# Reading in the data

In [1]:
# Import libraries
import pandas as pd

In [2]:
# Read in the data from the  huggingface library

splits = {'validation': 'en/validation-00000-of-00001.parquet', 'test': 'en/test-00000-of-00001.parquet', 'train': 'en/train-00000-of-00001.parquet'}
train = pd.read_parquet("hf://datasets/unimelb-nlp/wikiann/" + splits["train"])
val = pd.read_parquet("hf://datasets/unimelb-nlp/wikiann/" + splits["validation"])
test = pd.read_parquet("hf://datasets/unimelb-nlp/wikiann/" + splits["test"])

In [3]:
# Extract 2000 rows from the training data

subset=train[train.index % 10 == 0].copy()

In [4]:
# Convert each list of tokens to a readable string

sentences=[]
for index, row in subset.iterrows():
    sent=" ".join(row['tokens'])
    sent="# New sentence = "+sent
    sentences.append(sent)

In [5]:
# Add the sentence strings before each list of strings row in the dataframe

expanded=subset.copy()
for i, sentence in reversed(list(enumerate(sentences))):
    line=pd.DataFrame({'tokens':sentence, 'ner_tags':[0], 'langs':['en'], 'spans':['']})
    expanded=pd.concat([expanded.iloc[:i], line, expanded.iloc[i:]]).reset_index(drop=True)

In [6]:
all_tokens=expanded.explode('tokens')
toks=all_tokens['tokens']
# toks.to_csv('gathered_results.csv', index=False, encoding='UTF-8', sep=';')

# Dividing it between us

In [7]:
A=expanded[0:1000]
B=expanded[1000:2000]
C=expanded[2000:3000]
D=expanded[3000:4000]

In [8]:
Am=pd.concat([A, D])
Amina=Am.explode('tokens')
Amina['labels']=0
Amina=Amina[['tokens', 'labels']]
# Amina.to_csv('Amina_annotations.csv', index=False, encoding='UTF-8', sep=';')

In [9]:
Li=pd.concat([A,B])
Lilja=Li.explode('tokens')
Lilja['labels']=0
Lilja=Lilja[['tokens', 'labels']]
# Lilja.to_csv('Lilja_annotations.csv', index=False, encoding='UTF-8', sep=';')

In [10]:
Mi=pd.concat([B,C])
Miko=Mi.explode('tokens')
Miko['labels']=0
Miko=Miko[['tokens', 'labels']]
# Miko.to_csv('Mikolaj_annotations.csv', index=False, encoding='UTF-8', sep=';')

In [11]:
Zo=pd.concat([C,D])
Zosia=Zo.explode('tokens')
Zosia['labels']=0
Zosia=Zosia[['tokens', 'labels']]
# Zosia.to_csv('Zosia_annotations.csv', index=False, encoding='UTF-8', sep=';')

# Reading the results back in

In [12]:
results=pd.read_csv('results.csv', sep=';', encoding='utf-8')

In [13]:
results=results.fillna('0')
results['labels1']=results['labels1'].astype(int)
results['labels2']=results['labels2'].astype(int)
results['labels1']=results['labels1'].astype(str)
results['labels2']=results['labels2'].astype(str)

In [21]:
sentence=''
YAY=0
FALSEPOSNEG=0
ORGLOCERR=0
LOCSPLITERR=0
locspliterrlist=[]
OTHERERR=0
weird_results=[]


for index, row in results.iterrows():
    token=row.iloc[0]
    label1=row.iloc[1]
    label2=row.iloc[2]
    if token[0]=='#':
        sentence=token[17:]
        continue
    if label1 == label2:
        YAY +=1
    else:
        if ( ( label1 in ['3','4'] ) and (label2 in ['5','6'] ) ) or ( ( label2 in ['3','4'] ) and (label1 in ['5','6'] ) ):
            ORGLOCERR+=1
        elif ( label1=='0' and label2 in ['5','6'] ) or ( label2=='0' and label1 in ['5','6'] ) or (label1 in ['5', '6'] and label2 in ['5', '6']):
            LOCSPLITERR+=1
            locspliterrlist.append((sentence, token, label1, label2))
        elif label1=='0' or label2=='0':
            FALSEPOSNEG+=1
        else:
            OTHERERR+=1
            weird_results.append((sentence, token, label1, label2))

In [15]:
print("Correct results: ", YAY)
print("False positives or negatives: ", FALSEPOSNEG)
print("Disagreements of organisation or location: ", ORGLOCERR)
print("Location-splitting error (or perhaps just false negatives): ", LOCSPLITERR)
print("Nr of other unidentified errors: ", OTHERERR)

Correct results:  13881
False positives or negatives:  601
Disagreements of organisation or location:  323
Location-splitting error (or perhaps just false negatives):  805
Nr of other unidentified errors:  117


In [22]:
locspliterrlist

[('North Fanling New Town ( Extension of Fanling-Sheung Shui New Town )',
  'Fanling-Sheung',
  '0',
  '5'),
 ('North Fanling New Town ( Extension of Fanling-Sheung Shui New Town )',
  'Shui',
  '0',
  '6'),
 ('North Fanling New Town ( Extension of Fanling-Sheung Shui New Town )',
  'New',
  '0',
  '6'),
 ('North Fanling New Town ( Extension of Fanling-Sheung Shui New Town )',
  'Town',
  '0',
  '6'),
 ('Lăpugiu de Jos', 'Lăpugiu', '0', '5'),
 ('Lăpugiu de Jos', 'de', '0', '6'),
 ('Lăpugiu de Jos', 'Jos', '0', '6'),
 ('Borg El Arab Stadium , Alexandria , Egypt', 'Alexandria', '6', '5'),
 ('Borg El Arab Stadium , Alexandria , Egypt', 'Egypt', '6', '5'),
 ("'' Tsardom of Russia '' ' – Feodor III ( 1676–1682 )", 'Tsardom', '5', '0'),
 ("'' Tsardom of Russia '' ' – Feodor III ( 1676–1682 )", 'of', '6', '0'),
 ("'' Tsardom of Russia '' ' – Feodor III ( 1676–1682 )", 'Russia', '6', '0'),
 ('He was born in Clipston , Northamptonshire and died at Isleworth , Middlesex .',
  'Northamptonshire',

# Accuracy and Cohen's Kappa

In [17]:
# Function to calculate accuracy between two lists of labels
def calculate_accuracy(labels1, labels2):
    total_tokens = len(labels1)
    correct_tokens = sum(1 for label1, label2 in zip(labels1, labels2) if label1 == label2)
    accuracy = correct_tokens / total_tokens
    return accuracy

# Calculate accuracy
accuracy = calculate_accuracy(results['labels1'], results['labels2'])
print("Accuracy:", accuracy)

Accuracy: 0.8956394200936425


In [18]:
from collections import Counter

def cohen_kappa(annotation1, annotation2):
    #how much of each annotation
    count1 = Counter(annotation1)
    count2 = Counter(annotation2)
    
    # Observed agreement (P_o)
    observed_agreement = sum((a == b) for a, b in zip(annotation1, annotation2)) / len(annotation1)
    
    # Expected agreement (P_e)
    total = len(annotation1)
    categories = set(annotation1).union(set(annotation2)) # categories that appear in either of annotations
    expected_agreement = 0
    
    for category in categories:
        p1 = count1.get(category, 0) / total
        p2 = count2.get(category, 0) / total
        expected_agreement += p1 * p2
    
    # Cohen's Kappa calculation
    kappa = (observed_agreement - expected_agreement) / (1 - expected_agreement) #from the formula
    return kappa

# Calculate Cohen's Kappa
kappa = cohen_kappa(results['labels1'], results['labels2'])
print(f"Cohen's Kappa: {kappa}")

Cohen's Kappa: 0.7939666456275057


# F1 score

# Compare to silver data

In [20]:
silver_data=subset[['tokens', 'ner_tags']]
silver_data.head()

,tokens,ner_tags
0,"[R.H., Saunders, (, St., Lawrence, River, ), (...","[3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0]"
10,"[102.7, MHz, Triple, J, –, ABC]","[0, 0, 3, 4, 0, 3]"
20,"[Civic, Freedom, Party]","[3, 4, 4]"
30,"[Gian, Girolamo, Albani]","[1, 2, 2]"
40,"[North, Fanling, New, Town, (, Extension, of, ...","[5, 6, 6, 6, 0, 0, 0, 5, 6, 6, 6, 0]"


In [ ]:
for index, row in silver_data.iterrows():
    tokens=row.iloc[0]
    tags=row.iloc[1]
    for token, tag in zip(tokens, tags):
        